In [26]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import os

sys.path.append("../../note")
sys.path.append(os.getcwd())

%load_ext autoreload
%autoreload 2

from module.get_info_FinMind import FinMindClient
from module.get_info_Postgre import PostgreClient

START_DATE = '2010-01-01' # 資料起始年份
completed_list = []

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# 拿到台股總覽資料
stock_info = pd.read_csv('台股總覽.csv')
stock_info = stock_info.sort_values('股票代碼').reset_index(drop=True)
display(stock_info)
stock_info.to_csv('台股總覽.csv')

,產業別,股票代碼,股票名稱,市場別,更新日期
0,ETF,0050,元大台灣50,twse,2026-01-17
1,ETF,0051,元大中型100,twse,2026-01-17
2,ETF,0052,富邦科技,twse,2026-01-17
3,ETF,0053,元大電子,twse,2026-01-17
4,ETF,0054,元大台商50,twse,2026-01-16
...,...,...,...,...,...
4038,大盤,TAIEX,加權指數,twse,NaN
4039,大盤,TPEx,櫃買指數,tpex,NaN
4040,Index,Textiles,紡織纖維類指數,twse,NaN
4041,Index,Tourism,觀光事業類指數,twse,NaN


In [ ]:
# 從 finmind 下載並存入 postgresql
client = PostgreClient(system='windows', database='stock_daily')
fm = FinMindClient()

total_stocks = len(stock_info)

for i, row in tqdm(stock_info.iterrows(), total=total_stocks, desc="更新進度"):
    stock_id = str(row['股票代碼'])
    
    try:
        fm.initialize_frame(stock_id=stock_id, start_time=START_DATE, end_time=pd.Timestamp.now())
        
        df = fm.get_stock()
        
        if not df.empty:
            df = df.reset_index(drop=True)
            client.save_data(df, table=stock_id, if_exists='append', index=False)
            
        completed_list.append(stock_id)

    except Exception as e:
        print(f"處理 {stock_id} 時發生錯誤: {e}")

print("作業完成。")

TypeError: PostgreClient.__init__() missing 1 required positional argument: 'system'